In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Ключевые точки
# 2. Skeletal keypoints


In [ ]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils


def calculate_distance(point1, point2):
    return np.linalg.norm(np.array(point1) - np.array(point2))

def extract_keypoints(results, scale_factor=1.0):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

    # Применяем масштабирование к ключевым точкам
    pose[:len(pose)//4*3] *= scale_factor
    face *= scale_factor
    lh *= scale_factor
    rh *= scale_factor

    return np.concatenate([pose, face, lh, rh])


def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # BGR в RGB
    image.flags.writeable = False                  
    results = model.process(image)                 # Извлечение ключевых точек
    image.flags.writeable = True                   
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # RGB в BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Точки лица (404)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Точки скелета (33)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # левая рука (21)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # правая рука(21)

def draw_styled_landmarks(image, results):
    # лицо
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # скелет
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # левая рука
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # правая рука 
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 


# Захват видео
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        # Обработка изображения и извлечение ключевых точек
        image, results = mediapipe_detection(frame, holistic)
        if results.pose_landmarks:
            left_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER]
            right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER]
            shoulder_distance = calculate_distance(
                (left_shoulder.x, left_shoulder.y, left_shoulder.z),
                (right_shoulder.x, right_shoulder.y, right_shoulder.z)
            )

            # Коэффициент масштабирования
            scale_factor = 1.0 / shoulder_distance if shoulder_distance > 0 else 1.0

            keypoints = extract_keypoints(results, scale_factor=scale_factor)

        # Отображение изображения
        draw_landmarks(image, results)
        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
cap.release()
cv2.destroyAllWindows()

# 3. Сбор ключевых точек в массивы
# 3. Collect keypoints


In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# 4. Путь к данным, количество видео, длина последовательности
# 4. Data path, seq length param, numb of data param


In [ ]:
#Путь к папке
DATA_PATH = os.path.join('MP_Data') 

# классы действий
actions = np.array(['HARM', 'CHEST', 'SPEED'])

# Количество видео для сбора
no_sequences = 30

# Длина последовательности
sequence_length = 30

for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Функция сбора данных для обучения в режиме реального времени 
# 5. Real time data take function

In [ ]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):

                ret, frame = cap.read()
                image, results = mediapipe_detection(frame, holistic)
                draw_styled_landmarks(image, results)
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(1000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

# 5.2. Функция сбора данных из хранилища
# 5.2. Take data from storage function

In [ ]:
# Function to process each video file
def process_video(video_file, action, sequence_id):
    cap = cv2.VideoCapture(video_file)
    frame_num = 0
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened() and frame_num < sequence_length:
            ret, frame = cap.read()
            if not ret:
                break
            results = holistic.process(frame)
            draw_styled_landmarks(frame, results)

            if frame_num == 0:
                cv2.putText(frame, 'STARTING COLLECTION', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
            cv2.putText(frame, f'Collecting frames for {action} Video Number {sequence_id}', (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
            cv2.imshow('OpenCV Feed', frame)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
            
            keypoints = extract_keypoints(results)
            print(keypoints.shape)
            npy_path = os.path.join(DATA_PATH, action, str(sequence_id), str(frame_num))
            np.save(npy_path, keypoints)
            frame_num += 1

        cap.release()
        cv2.destroyAllWindows()

# Example usage
video_files = {
    'HARM': [''],
    'CHEST': [''], 
    'SPEED': [''], 
}

# Process each video for each actiod
for action, videos in video_files.items():
    for idx, video_file in enumerate(videos):
        if idx < no_sequences:
            process_video(video_file, action, idx)


# 6. Модуль подготовки данных
# 6. data transformation part



In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
label_map = {label:num for num, label in enumerate(actions)}
label_map

sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

np.array(sequences).shape

np.array(labels).shape

X = np.array(sequences)

X.shape

y = to_categorical(labels).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

from sklearn.model_selection import train_test_split

# Разделение данных на тренировочные и тестовые
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Дополнительное разделение тренировочной выборки на тренировочную и валидационную
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.8, random_state=42)

# Теперь у тебя есть X_train, X_val, X_test, y_train, y_val, y_test


In [ ]:
y_test.shape

# 7. Инициализация модели RNN
# 7. Model init

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

model.summary()

# 8. Проверка модели
# 8. Model Check

In [ ]:
res = model.predict(X_test)

In [ ]:
actions[np.argmax(res[3])]

In [ ]:
actions[np.argmax(y_test[4])]

# 9. Сохранение весов
# 9. Save the weights

In [ ]:
model.save('action.h5')

In [ ]:
del model

In [ ]:
model.load_weights('action.h5')

# 10. Проверка модели
# 10. Check model

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, confusion_matrix

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

# 11. Модуль распознавания действий в реальном времени
# 11. Real time recognition part

In [ ]:
from PIL import Image, ImageDraw, ImageFont
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,100+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

def display_current_class(current_class, input_frame, color=(255, 255, 255)):
    output_frame = input_frame.copy()
    # Display the current class at the top of the frame
    cv2.rectangle(output_frame, (0, 0), (640, 40), (245, 117, 16), -1)
    cv2.putText(output_frame, current_class, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
    return output_frame

def draw_text_pil(img, text, position, font, font_scale, color):
    img_pil = Image.fromarray(img)
    draw = ImageDraw.Draw(img_pil)
    font = ImageFont.truetype(font, font_scale)
    draw.text(position, text, font=font, fill=color)
    return np.array(img_pil)

def split_text(text, max_width, font, font_scale):
    font = ImageFont.truetype(font, font_scale)
    words = text.split()
    lines = []
    line = ""
    for word in words:
        if font.getsize(line + word)[0] <= max_width:
            line = line + word + " "
        else:
            lines.append(line)
            line = word + " "
    lines.append(line)
    return lines


# Захват видео
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        # Обработка изображения и извлечение ключевых точек
        image, results = mediapipe_detection(frame, holistic)
        if results.pose_landmarks:
            left_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER]
            right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER]
            shoulder_distance = calculate_distance(
                (left_shoulder.x, left_shoulder.y, left_shoulder.z),
                (right_shoulder.x, right_shoulder.y, right_shoulder.z)
            )

            # Коэффициент масштабирования
            scale_factor = 1.0 / shoulder_distance if shoulder_distance > 0 else 1.0

            keypoints = extract_keypoints(results, scale_factor=scale_factor)

        # Отображение изображения
        draw_landmarks(image, results)
        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


import cv2
import numpy as np
import mediapipe as mp
from PIL import Image, ImageDraw, ImageFont

# 1. New detection variables
sequence = []
sentence = []
threshold = 0.7

# Словарь рекомендаций
recommendations = {
    'HARM': 'ПОДНИМАЙТЕ ШТАНГУ НИЖЕ ГРУДИ',
    'CHEST': 'ДЕРЖИТЕ СПИНУ РОВНО, НЕ СГИБАЙТЕ КОЛЕНИ',
    'SPEED': 'ВЫПОЛНЯЙТЕ УПРАЖНЕНИЕ МЕДЛЕНЕЕ, ОПТИМАЛЬНОЕ ВРЕМЯ 3-4 СЕКУНДЫ',
    # Добавьте столько классов и рекомендаций, сколько необходимо
}

# Функция для рисования текста с использованием PIL
def draw_text_pil(img, text, position, font_path, font_scale, color):
    img_pil = Image.fromarray(img)
    draw = ImageDraw.Draw(img_pil)
    font = ImageFont.truetype(font_path, font_scale)
    draw.text(position, text, font=font, fill=color)
    return np.array(img_pil)

# Функция для разделения текста на строки
def split_text(text, max_width, font_path, font_scale):
    font = ImageFont.truetype(font_path, font_scale)
    words = text.split()
    lines = []
    line = ""
    for word in words:
        if font.getsize(line + word)[0] <= max_width:
            line = line + word + " "
        else:
            lines.append(line)
            line = word + " "
    lines.append(line)
    return lines

cap = cv2.VideoCapture('')
#cap = cv2.VideoCapture(0)
# Set mediapipe model 
mp_holistic = mp.solutions.holistic
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()
        if not ret:
            break

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-60:]
        
        if len(sequence) == 60:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            # 3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Получаем текущий класс и рекомендацию
            current_action = actions[np.argmax(res)]
            recommendation = recommendations.get(current_action, "No recommendation available")

        max_width = 640  # Максимальная ширина текста на изображении
        lines = split_text(' '.join(sentence), max_width, 'arial.ttf', 24)
        
        # Разделим строку класса и рекомендацию на разные области
        cv2.rectangle(image, (0, 0), (max_width, 40), (245, 117, 16), -1)  # Полоса сверху для класса
        image = draw_text_pil(image, ' '.join(sentence), (3, 10), 'arial.ttf', 24, (255, 255, 255))

        # Отображение рекомендации в центре экрана
        rec_lines = split_text(recommendation, max_width, 'arial.ttf', 24)
        y = int((image.shape[0] - (len(rec_lines) * 30)) / 8)
        for line in rec_lines:
            image = draw_text_pil(image, line, (3, y), 'arial.ttf', 24, (255, 255, 255))
            y += 30

        scale_factor = 1.3  # Коэффициент увеличения
        width = int(image.shape[1] * scale_factor)
        height = int(image.shape[0] * scale_factor)
        dim = (width, height)
        resized = cv2.resize(image, dim, interpolation=cv2.INTER_LINEAR)
        
        # Show to screen
        cv2.imshow('ТРЕНЕР', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()



def calculate_distance(point1, point2):
    return np.linalg.norm(np.array(point1) - np.array(point2))

def extract_keypoints(results, scale_factor=1.0):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

    # Применяем масштабирование к ключевым точкам
    pose[:len(pose)//4*3] *= scale_factor
    face *= scale_factor
    lh *= scale_factor
    rh *= scale_factor

    return np.concatenate([pose, face, lh, rh])


In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
model.predict(np.expand_dims(X_test[2], axis=0))

In [ ]:
X_test[0]